# Gaussian Proccess (GP) on M1 and S2 Site Data
We wish to determine the seasonal and diurnal cycles of supermicron aerosols/bioaerosols. In this notebook we focus on the seasonal trends. We will split the data up by seasons and fit a GP on a subset of the data. Using a confidence interval, we will determine if the data indicates seasonal cycles. 

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal
from scipy.spatial.distance import cdist
from scipy.special import kv, gamma
import itertools
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets
from ipywidgets import interactive

os.environ["OMP_NUM_THREADS"] = "2"



def data_import(path1, delimiter):
    #locate file path and import data
    if delimiter == 'none':
        file1 = pd.read_csv(path1)
    else:   
        file1 = pd.read_csv(path1, delimiter=delimiter)
    return file1

def data_convert(file1, time_column_name: str):
    #convert timestamps to datetime format
    file1[time_column_name] = pd.to_datetime(file1[time_column_name])
   
    #handle missing values (NaN); fill with mean value
    file1.fillna(file1.mean(), inplace=True)

    return file1

def data_frequency(file1, desired_frequency: str, time_column_name: str):
    if desired_frequency == 'ten_minute':
        file1 = file1.resample('10T', on=time_column_name).mean()
    elif desired_frequency == '4_hourly':
        file1 = file1.resample('4h', on=time_column_name).mean()
    elif desired_frequency == 'hourly':
        file1 = file1.resample('h', on=time_column_name).mean()
    elif desired_frequency == 'twelve_hourly':
        file1 = file1.resample('12h', on=time_column_name).mean()
    elif desired_frequency == 'daily':
        file1 = file1.resample('D', on=time_column_name).mean()

    # Ensure the index is datetime
    file1.index = pd.to_datetime(file1.index)
    
    # Reset the index and name it 'Time(UTC)'
    file1.reset_index(inplace=True)
    file1.rename(columns={file1.index.name: time_column_name}, inplace=True)

    return file1


def time_to_sincos(df):
    sin_values = []
    cos_values = []

    for i in range(len(df)):
        sin_values.append(np.sin((2 * np.pi * i) / 365.25))
        cos_values.append(np.cos((2 * np.pi * i) / 365.25))
        
    df['Time_sin'] = sin_values
    df['Time_cos'] = cos_values
    
    return df

#trim data to desired collumns
def file_trim(file1, desired_collumns):
    file1 = file1[desired_collumns]
    return file1

#separate data by seasons
def data_split(file1, time_column_name: str):
    winter = file1[(file1[time_column_name] >= '2022-12-21') & (file1[time_column_name] < '2023-03-20')]
    spring = file1[(file1[time_column_name] >= '2023-03-20') & (file1[time_column_name] < '2023-06-21')]
    summer = file1[(file1[time_column_name] >= '2022-06-21') & (file1[time_column_name] < '2022-09-23')]
    autunm = file1[(file1[time_column_name] >= '2022-09-23') & (file1[time_column_name] < '2022-12-21')]
    return winter, spring, summer, autunm





# S2 site data importing and modeling 
Here we import and handle all of the data. s2 first, split the varying frequencies and seasons, then M1 data

In [ ]:
#importing s2 and m1 data sets
s2_site_data= data_import("C:\\Users\\396760\\lanl\\data\\ARMSAILS2_cleaned.csv", 'none')
m1_site_data = data_import("C:\\Users\\396760\\lanl\\data\\ARMSAILM1_cleaned.csv", 'none')

#data conversion and handling for m1, s2 data
s2_site_data, m1_site_data = data_convert(s2_site_data, 'Time(UTC)'), data_convert(m1_site_data, 'Time(UTC)')

#defining collumns and imp feature variables
collumns = ['Time(UTC)', 'sample_rh_pct', 'sample_temp_C', 'pm_1_ug_per_m3', 'Time_sin', 'Time_cos']
collumns1 = ['Time(UTC)', 'sample_rh_pct', 'sample_temp_C', 'pm_25_ug_per_m3', 'Time_sin', 'Time_cos']
features = ['sample_rh_pct', 'sample_temp_C', 'Time_sin', 'Time_cos']
s2_target_pm1 = m1_target_pm1 = ['pm_1_ug_per_m3']
s2_target_pm25 = m1_target_pm25 = ['pm_25_ug_per_m3']
s2_target_pm10 = m1_target_pm10 = ['pm_10_ug_per_m3']

In [ ]:
#defining datasets of differing frequencies in the following order : daily, 12 hourly, 4 hourly and hourly, 
s2_site_daily, m1_site_daily = data_frequency(s2_site_data, 'daily', 'Time(UTC)'), data_frequency(m1_site_data, 'daily', 'Time(UTC)')
s2_site_12hourly, m1_site_12hourly = data_frequency(s2_site_data, 'twelve_hourly', 'Time(UTC)'), data_frequency(m1_site_data, 'twelve_hourly', 'Time(UTC)')
s2_site_4hourly, m1_site_4hourly = data_frequency(s2_site_data, '4_hourly', 'Time(UTC)'), data_frequency(m1_site_data, '4_hourly', 'Time(UTC)')
s2_site_hourly, m1_site_hourly = data_frequency(s2_site_data, 'hourly', 'Time(UTC)'), data_frequency(m1_site_data, 'hourly', 'Time(UTC)')

#time to sin cos conversion
s2_site_daily, m1_site_daily = time_to_sincos(s2_site_daily), time_to_sincos(m1_site_daily)
s2_site_12hourly, m1_site_12hourly = time_to_sincos(s2_site_12hourly), time_to_sincos(m1_site_12hourly)
s2_site_4hourly, m1_site_4hourly = time_to_sincos(s2_site_4hourly), time_to_sincos(m1_site_4hourly)
s2_site_hourly, m1_site_hourly = time_to_sincos(s2_site_hourly), time_to_sincos(m1_site_hourly)

#trimming data to desired collumns
s2_site_daily, m1_site_daily = file_trim(s2_site_daily, collumns), file_trim(m1_site_daily, collumns)
s2_site_12hourly, m1_site_12hourly = file_trim(s2_site_12hourly, collumns), file_trim(m1_site_12hourly, collumns)
s2_site_4hourly, m1_site_4hourly = file_trim(s2_site_4hourly, collumns), file_trim(m1_site_4hourly, collumns)
s2_site_hourly, m1_site_hourly = file_trim(s2_site_hourly, collumns), file_trim(m1_site_hourly, collumns)

#splitting data by seasons

#daily
s2_winter_daily, s2_spring_daily, s2_summer_daily, s2_autunm_daily = data_split(s2_site_daily, 'Time(UTC)')
m1_winter_daily, m1_spring_daily, m1_summer_daily, m1_autunm_daily = data_split(m1_site_daily, 'Time(UTC)')
#12 hourly
s2_winter_12hourly, s2_spring_12hourly, s2_summer_12hourly, s2_autunm_12hourly = data_split(s2_site_12hourly, 'Time(UTC)')
m1_winter_12hourly, m1_spring_12hourly, m1_summer_12hourly, m1_autunm_12hourly = data_split(m1_site_12hourly, 'Time(UTC)')
#4 hourly
s2_winter_4hourly, s2_spring_4hourly, s2_summer_4hourly, s2_autunm_4hourly = data_split(s2_site_4hourly, 'Time(UTC)')
m1_winter_4hourly, m1_spring_4hourly, m1_summer_4hourly, m1_autunm_4hourly = data_split(m1_site_4hourly, 'Time(UTC)')
#hourly
s2_winter_hourly, s2_spring_hourly, s2_summer_hourly, s2_autunm_hourly = data_split(s2_site_hourly, 'Time(UTC)')
m1_winter_hourly, m1_spring_hourly, m1_summer_hourly, m1_autunm_hourly = data_split(m1_site_hourly, 'Time(UTC)')

#plots for s2 data sites at varying frequencies (PM1)

#daily
plt.figure(figsize=(10,5))
plt.plot(s2_winter_daily['Time(UTC)'], s2_winter_daily['pm_1_ug_per_m3'], label='Winter')
plt.plot(s2_spring_daily['Time(UTC)'], s2_spring_daily['pm_1_ug_per_m3'], label='Spring')
plt.plot(s2_summer_daily['Time(UTC)'], s2_summer_daily['pm_1_ug_per_m3'], label='Summer')
plt.plot(s2_autunm_daily['Time(UTC)'], s2_autunm_daily['pm_1_ug_per_m3'], label='Autunm')
plt.legend()
plt.show()

#12 hourly
plt.figure(figsize=(10,5))
plt.plot(s2_winter_12hourly['Time(UTC)'], s2_winter_12hourly['pm_1_ug_per_m3'], label='Winter')
plt.plot(s2_spring_12hourly['Time(UTC)'], s2_spring_12hourly['pm_1_ug_per_m3'], label='Spring')
plt.plot(s2_summer_12hourly['Time(UTC)'], s2_summer_12hourly['pm_1_ug_per_m3'], label='Summer')
plt.plot(s2_autunm_12hourly['Time(UTC)'], s2_autunm_12hourly['pm_1_ug_per_m3'], label='Autunm')
plt.legend()
plt.show()

#4 hourly
plt.figure(figsize=(10,5))
plt.plot(s2_winter_4hourly['Time(UTC)'], s2_winter_4hourly['pm_1_ug_per_m3'], label='Winter')
plt.plot(s2_spring_4hourly['Time(UTC)'], s2_spring_4hourly['pm_1_ug_per_m3'], label='Spring')
plt.plot(s2_summer_4hourly['Time(UTC)'], s2_summer_4hourly['pm_1_ug_per_m3'], label='Summer')
plt.plot(s2_autunm_4hourly['Time(UTC)'], s2_autunm_4hourly['pm_1_ug_per_m3'], label='Autunm')
plt.legend()
plt.show()

#hourly
plt.figure(figsize=(10,5))
plt.plot(s2_winter_hourly['Time(UTC)'], s2_winter_hourly['pm_1_ug_per_m3'], label='Winter')
plt.plot(s2_spring_hourly['Time(UTC)'], s2_spring_hourly['pm_1_ug_per_m3'], label='Spring')
plt.plot(s2_summer_hourly['Time(UTC)'], s2_summer_hourly['pm_1_ug_per_m3'], label='Summer')
plt.plot(s2_autunm_hourly['Time(UTC)'], s2_autunm_hourly['pm_1_ug_per_m3'], label='Autunm')
plt.legend()
plt.show()



visualizing our choice of kernel

In [ ]:
def visualize_kernel_function_space(kernel, x_range, n_functions, **kernel_params):
    # Create a grid of x values
    X = np.atleast_2d(x_range).T

    # Compute the kernel matrix
    K = kernel(X, X, **kernel_params)

    # Clear the plot
    plt.figure(figsize=(10, 6))
    
    # Generate functions
    for _ in range(n_functions):
        # Draw samples from a multivariate normal distribution
        f = multivariate_normal.rvs(mean=10*np.ones(X.shape[0]), cov=K)
        
        # Plot the function
        plt.plot(x_range, f)

    plt.xlabel('X')
    plt.ylabel('Function Value')
    plt.title(f'Functions Sampled from {kernel.__name__}')
    plt.grid(True)
    plt.show()


def white_noise_kernel(x1, x2, variance=5.0):
    # Compute the kernel
    if np.array_equal(x1, x2):
        return variance * np.eye(x1.shape[0])
    else:
        return np.zeros((x1.shape[0], x2.shape[0]))
    
def rbf_kernel(x1, x2, length_scale=1.0, variance=1.0):
    # Compute the pairwise squared distances between the inputs
    sq_dists = cdist(x1, x2, 'sqeuclidean')
    
    # Compute the kernel
    return variance * np.exp(-0.5 * sq_dists / length_scale**2)

def periodic_kernel(x1, x2, length_scale=1.0, variance=1.0, period=1.0):
    # Compute the pairwise squared distances between the inputs
    sq_dists = cdist(x1, x2, 'sqeuclidean')
    
    # Compute the kernel
    return variance * np.exp(-2 * np.sin(np.pi * np.sqrt(sq_dists) / period)**2 / length_scale**2)

def matern_kernel(x, x_star, length_scale=1.0, nu=0.5, variance=1.0):
    dists = cdist(x / length_scale, x_star / length_scale, metric='euclidean')
    
    if nu == 0.5:
        K = np.exp(-dists)
    elif nu == 1.5:
        sqrt3 = np.sqrt(3)
        K = (1.0 + sqrt3 * dists) * np.exp(-sqrt3 * dists)
    elif nu == 2.5:
        sqrt5 = np.sqrt(5)
        K = (1.0 + sqrt5 * dists + (5.0 / 3.0) * (dists ** 2)) * np.exp(-sqrt5 * dists)

    return variance * K

def sum_kernel(x1, x2, variance, length_scale, nu, period):
    return white_noise_kernel(x1, x2, variance=variance) + matern_kernel(x1, x2, variance=variance, length_scale=length_scale, nu=nu) + periodic_kernel(x1, x2, variance=variance, length_scale=length_scale, period=period)

x_space = np.linspace(0, 10, 100)
num_of_functions = 3

# Create sliders for kernel parameters
def plot_white_noise_kernel(variance):
    visualize_kernel_function_space(white_noise_kernel, x_space, num_of_functions, variance=variance)

def plot_rbf_kernel(length_scale, variance):
    visualize_kernel_function_space(rbf_kernel, x_space, num_of_functions, length_scale=length_scale, variance=variance)

def plot_periodic_kernel(length_scale, variance, period):
    visualize_kernel_function_space(periodic_kernel, x_space, num_of_functions, length_scale=length_scale, variance=variance, period=period)

def plot_matern_kernel(length_scale, nu, variance):
    visualize_kernel_function_space(matern_kernel, x_space, num_of_functions, length_scale=length_scale, nu=nu, variance=variance)

def plot_sum_kernel(variance, length_scale, nu, period):
    visualize_kernel_function_space(sum_kernel, x_space, num_of_functions, variance=variance, length_scale=length_scale, nu=nu, period=period)

# Create interactive widgets
interactive_plot_white_noise = interactive(plot_white_noise_kernel, variance=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=2.0))
interactive_plot_rbf = interactive(plot_rbf_kernel, length_scale=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0), variance=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0))
interactive_plot_periodic = interactive(plot_periodic_kernel, length_scale=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0), variance=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0), period=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0))
interactive_plot_matern = interactive(plot_matern_kernel, length_scale=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0), nu=widgets.FloatSlider(min=0.5, max=2.5, step=1.0, value=0.5), variance=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0))
interactive_plot_sum = interactive(plot_sum_kernel, variance=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0), length_scale=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0), nu=widgets.FloatSlider(min=0.5, max=2.5, step=1.0, value=0.5), period=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=1.0))
display(interactive_plot_white_noise)
display(interactive_plot_rbf)
display(interactive_plot_periodic)
display(interactive_plot_matern)
display(interactive_plot_sum)


In [ ]:
def extract_subset(X, y, n):

    # Number of samples to select
    num_samples = int((n / 100) * X.shape[0])

    # Generate a random subset of indices
    subset_indices = np.random.choice(X.shape[0], num_samples, replace=False)

    # Extract the subset of the data, for y
    X_subset = X[subset_indices, :]
    y_subset = y[subset_indices, :]

   

    return X_subset, y_subset

def covariance_matrix(x, x_star, kernel, **kernel_params):
    return kernel(x, x_star, **kernel_params)

def run_gp_regression(X, y, X_star, kernel, **kernel_params):

    # Compute the covariance matrices
    K = covariance_matrix(X, X, kernel, **kernel_params)
    K_star = covariance_matrix(X, X_star, kernel, **kernel_params)
    K_star_star = covariance_matrix(X_star, X_star, kernel, **kernel_params)
    
    # Compute the Cholesky decomposition of K
    L = np.linalg.cholesky(K + 1e-6 * np.eye(len(X)))
    
    # Compute the mean of the posterior predictive distribution
    alpha = np.linalg.solve(L.T, np.linalg.solve(L, y))
    f_star_mean = K_star.T @ alpha
    
    # Compute the variance of the posterior predictive distribution
    v = np.linalg.solve(L, K_star)
    f_star_var = K_star_star - v.T @ v
    
    return f_star_mean, f_star_var

def RMSE_calc_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

#plot 2d gp with x-axis as Time(UTC) dates and y axis as pm_1_ug_per_m3 values
def plot_gp_regression(season, X, y, X_star, y_star, f_star_mean, f_star_var, kernel, **kernel_params):
    
    # Plot the true data
    plt.figure(figsize=(10, 6))
    plt.scatter(X, y, color='black', label='true data')
    
    # Plot the training/test points (subset of true data)
    plt.plot(X_star, y_star, color='blue', label='True Function')
    
    # Plot the mean of the posterior predictive distribution
    plt.plot(X_star, f_star_mean, color='red', label='Posterior Mean')
    
    # Plot the 95% confidence interval
    f_star_std = np.sqrt(np.diag(f_star_var)).reshape(-1,1)  # Extracting standard deviation
    plt.fill_between(X_star.ravel(), 
                    (f_star_mean - 1.96 * f_star_std).ravel(), 
                    (f_star_mean + 1.96 * f_star_std).ravel(), 
                     color='red', alpha=0.2, label='95% Confidence Interval')
    
    plt.xlabel(season)
    plt.ylabel('pm_1_ug_per_m3')
    plt.title(f'GP Regression with {kernel.__name__}')
    plt.legend()
    plt.grid(True)
    plt.show()


running the gp with various datasets

In [ ]:
def extract_run_gp_RSME_plot(season, X, y, n, x_axis_feature, y_axis_value, kernel, **kernel_params):
    X_subset, y_subset = extract_subset(X, y, n)

    f_mean_star, f_var_star = run_gp_regression(X_subset, y_subset, X, kernel, **kernel_params)
    
    #print RSME
    print(f'RMSE: {RMSE_calc_rmse(y_axis_value.values, f_mean_star)}')

    plot_gp_regression(season, x_axis_feature.values, y_axis_value.values, x_axis_feature.values , y_axis_value.values, f_mean_star, f_var_star, kernel, **kernel_params)



""" #extracting and plotting gp regression for s2_{season}_daily data set, autumn, winter, summer, spring
n = 30
X = s2_autunm_daily[features].values
y = s2_autunm_daily['pm_1_ug_per_m3'].values
X, y = np.array(X), y.reshape(-1, 1)

# sum kernel
extract_run_gp_RSME_plot('autumn', X, y, n, s2_autunm_daily['Time(UTC)'], s2_autunm_daily['pm_1_ug_per_m3'], sum_kernel, length_scale=2.5, nu=1.5, variance=5.0, period=1.0)

X = s2_winter_daily[features].values
y = s2_winter_daily['pm_1_ug_per_m3'].values
X, y = np.array(X), y.reshape(-1, 1)

extract_run_gp_RSME_plot('winter', X, y, n, s2_winter_daily['Time(UTC)'], s2_winter_daily['pm_1_ug_per_m3'], sum_kernel, length_scale=2.5, nu=1.5, variance=5.0, period=1.0)

X = s2_summer_daily[features].values
y = s2_summer_daily['pm_1_ug_per_m3'].values
X, y = np.array(X), y.reshape(-1, 1)

extract_run_gp_RSME_plot('summer', X, y, n, s2_summer_daily['Time(UTC)'], s2_summer_daily['pm_1_ug_per_m3'], sum_kernel, length_scale=2.5, nu=1.5, variance=1.0, period=1.0)

X = s2_spring_daily[features].values
y = s2_spring_daily['pm_1_ug_per_m3'].values
X, y = np.array(X), y.reshape(-1, 1)

extract_run_gp_RSME_plot('spring', X, y, n, s2_spring_daily['Time(UTC)'], s2_spring_daily['pm_1_ug_per_m3'], sum_kernel, length_scale=2.5, nu=1.5, variance=5.0, period=1.0) """




